In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import beta

# Загрузка данных
data_url = 'https://raw.githubusercontent.com/aaubs/data_science_master/main/data/ab_data.csv'
df = pd.read_csv(data_url)

# Преобразование столбца timestamp в тип datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Группировка данных по часам
grouped_data = df.groupby(df['timestamp'].dt.floor('H'))

# Создание пустых списков для хранения результатов
win_probabilities_b = []

# Инициализация априорного распределения
alpha_prior = 1  # Параметр априорного распределения
beta_prior = 1   # Параметр априорного распределения

# Итерация по группам данных
for timestamp, group in grouped_data:
    # Разделение данных на группы A и B
    group_a = group[group['group'] == 'A']
    group_b = group[group['group'] == 'B']

    # Вычисление количества успехов и неудач для каждой группы
    success_a = group_a['converted'].sum()
    success_b = group_b['converted'].sum()
    failures_a = len(group_a) - success_a
    failures_b = len(group_b) - success_b

    # Обновление вероятностей для группы A
    alpha_posterior_a = alpha_prior + success_a
    beta_posterior_a = beta_prior + failures_a

    # Обновление вероятностей для группы B
    alpha_posterior_b = alpha_prior + success_b
    beta_posterior_b = beta_prior + failures_b

    # Генерация случайных выборок из бета-распределения
    samples_a = beta.rvs(alpha_posterior_a, beta_posterior_a, size=40000)
    samples_b = beta.rvs(alpha_posterior_b, beta_posterior_b, size=40000)

    # Расчет вероятности победы группы B
    win_probability_b = (samples_b > samples_a).mean()

    # Добавление вероятности победы группы B в список
    win_probabilities_b.append(win_probability_b)

    # Обновление априорного распределения для следующей итерации
    alpha_prior = alpha_posterior_b
    beta_prior = beta_posterior_b

    # Вывод результатов по каждому часу
    print(f'Время: {timestamp}')
    print(f'Вероятность победы группы B: {win_probability_b:.4f}\n')

# Вывод средней вероятности победы группы B на протяжении всех часов
print(f'Средняя вероятность победы группы B: {np.mean(win_probabilities_b):.4f}')